In [1]:
from matplotlib.cm import get_cmap
from matplotlib.axes import Axes

In [2]:
import librosa
import librosa.display
import numpy as np
import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import regularizers
from sklearn.utils import shuffle
import pandas as pd
import glob
import matplotlib.pyplot as plt
import scipy.io.wavfile
import sys
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
#from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.model_selection import train_test_split
import os
import json

In [3]:
root = "D:/College Work/Internship/IIIT-A/Code/Audio_Trimmed_Videos/"

In [4]:
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(root):
    for file in f:
        files.append(file)

In [5]:
descriptors = pd.read_csv('Descriptors.csv', index_col=0)
descriptors.head()

,gender,gender_no,output,expression_index,video_name,start_time_in_sec,end_time_in_sec,start_exp,end_exp,text_file,kinect_start_frame,kinect_end_frame
1,F,3,An,1,927_0216_01.MP4,5.356700,9.481139,161,284,20170902_092359_00.txt,217,312
2,F,3,An,2,927_0216_01.MP4,9.883585,14.903150,296,447,20170902_092359_00.txt,322,466
3,F,3,An,3,927_0216_01.MP4,14.903150,19.427206,447,582,20170902_092359_00.txt,466,564
4,F,3,An,4,927_0216_01.MP4,19.427206,25.077029,582,752,20170902_092359_00.txt,564,699
5,F,3,An,5,927_0216_01.MP4,25.077029,29.981168,752,899,20170902_092359_00.txt,699,828


In [69]:
files[0]

'927_0196_01-1.wav'

In [6]:
name = str(files[0])[:-6] + '.MP4'
name

'927_0196_01.MP4'

In [74]:
emotion = descriptors.loc[descriptors['video_name']==name]['output'][:1].values[0]

In [75]:
emotion

'Ha'

In [76]:
mylist = []
for aud in files:
    name = str(aud)[:-6] + '.MP4'
    emotion = descriptors.loc[descriptors['video_name']==name]['output'][:1].values[0]
    mylist.append(emotion)

In [78]:
len(mylist

315

In [10]:
feeling_list=[]
for item in mylist:
    if item=='Ne':
        feeling_list.append(1)
    elif item=='Sa':
        feeling_list.append(2)
    elif item=='Su':
        feeling_list.append(3)
    elif item=='Fe':
        feeling_list.append(4)
    elif item=='An':
        feeling_list.append(5)
    elif item=='Di':
        feeling_list.append(6)
    elif item=='Ha':
        feeling_list.append(7)

labels = pd.DataFrame(feeling_list)

In [11]:
files[200:223]

['927_0246_01-1.wav',
 '927_0246_01-2.wav',
 '927_0246_01-3.wav',
 '927_0246_01-4.wav',
 '927_0246_01-5.wav',
 '927_0247_01-1.wav',
 '927_0247_01-2.wav',
 '927_0247_01-3.wav',
 '927_0247_01-4.wav',
 '927_0247_01-5.wav',
 '927_0248_01-1.wav',
 '927_0248_01-2.wav',
 '927_0248_01-3.wav',
 '927_0248_01-4.wav',
 '927_0248_01-5.wav',
 '927_0249_01-1.wav',
 '927_0249_01-2.wav',
 '927_0249_01-3.wav',
 '927_0249_01-4.wav',
 '927_0249_01-5.wav',
 '927_0250_01-1.wav',
 '927_0250_01-2.wav',
 '927_0250_01-3.wav']

In [12]:
feeling_list[200:223]

[6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3]

In [13]:
df = pd.DataFrame(columns=['feature'])
bookmark = 0
for audio in files:
    X, sample_rate = librosa.load(root + audio, res_type='kaiser_fast',duration=None ,sr=None,offset=0.0)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=18), axis=0)
    feature = mfccs

    df.loc[bookmark] =[feature]
    bookmark=bookmark+1


In [14]:
df

,feature
0,"[-22.576344, -22.63391, -22.336678, -22.73294,..."
1,"[-16.56112, -15.109053, -14.493184, -13.463159..."
2,"[-27.23587, -26.376987, -24.637917, -25.186186..."
3,"[-25.14695, -23.783045, -24.60435, -24.942022,..."
4,"[-21.970781, -21.76817, -20.904003, -20.710114..."
...,...
310,"[-15.110401, -16.126104, -16.642399, -17.01920..."
311,"[-10.852483, -13.523667, -14.5473, -16.2716, -..."
312,"[-11.88979, -13.000632, -16.028648, -17.109066..."
313,"[-25.888529, -22.996035, -22.643534, -22.62645..."


In [15]:
print(len(df['feature'][0]))

257


In [16]:
idf3 = pd.DataFrame(df['feature'].values.tolist())

In [17]:
inewdf = pd.concat([idf3,labels], axis=1)

In [18]:
inewdf.index

RangeIndex(start=0, stop=315, step=1)

In [19]:
irnewdf = inewdf.rename(index=str, columns={0: "label"})

In [20]:
irnewdf.index

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '305', '306', '307', '308', '309', '310', '311', '312', '313', '314'],
      dtype='object', length=315)

In [21]:
irnewdf

,label,1,2,3,4,5,6,7,8,9,...,1046,1047,1048,1049,1050,1051,1052,1053,1054,label
0,-22.576344,-22.633909,-22.336678,-22.732941,-21.356634,-21.403225,-22.669485,-20.614010,-21.624844,-21.599819,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,-16.561119,-15.109053,-14.493184,-13.463159,-13.639797,-14.605306,-15.306802,-14.486601,-13.109497,-13.043911,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,-27.235870,-26.376987,-24.637917,-25.186186,-25.912806,-23.534084,-22.861095,-23.740396,-24.684076,-23.722967,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,-25.146950,-23.783045,-24.604349,-24.942022,-24.671974,-22.716316,-21.737007,-22.425535,-23.134193,-23.291124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,-21.970781,-21.768169,-20.904003,-20.710114,-22.251064,-24.140152,-22.357712,-21.901991,-23.199156,-21.536785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,-15.110401,-16.126104,-16.642399,-17.019207,-17.975000,-18.825453,-19.487116,-19.043617,-20.585655,-22.298397,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
311,-10.852483,-13.523667,-14.547300,-16.271601,-16.924763,-17.488625,-18.852144,-20.408836,-20.884806,-21.933760,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
312,-11.889790,-13.000632,-16.028648,-17.109066,-15.882111,-15.968800,-16.425117,-18.963812,-19.968313,-17.888788,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
313,-25.888529,-22.996035,-22.643534,-22.626450,-21.710609,-22.616358,-21.866781,-19.947834,-19.194817,-20.434992,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7


In [22]:
irnewdf = irnewdf.fillna(0)

In [23]:
irnewdf

,label,1,2,3,4,5,6,7,8,9,...,1046,1047,1048,1049,1050,1051,1052,1053,1054,label
0,-22.576344,-22.633909,-22.336678,-22.732941,-21.356634,-21.403225,-22.669485,-20.614010,-21.624844,-21.599819,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,-16.561119,-15.109053,-14.493184,-13.463159,-13.639797,-14.605306,-15.306802,-14.486601,-13.109497,-13.043911,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,-27.235870,-26.376987,-24.637917,-25.186186,-25.912806,-23.534084,-22.861095,-23.740396,-24.684076,-23.722967,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,-25.146950,-23.783045,-24.604349,-24.942022,-24.671974,-22.716316,-21.737007,-22.425535,-23.134193,-23.291124,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,-21.970781,-21.768169,-20.904003,-20.710114,-22.251064,-24.140152,-22.357712,-21.901991,-23.199156,-21.536785,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,-15.110401,-16.126104,-16.642399,-17.019207,-17.975000,-18.825453,-19.487116,-19.043617,-20.585655,-22.298397,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
311,-10.852483,-13.523667,-14.547300,-16.271601,-16.924763,-17.488625,-18.852144,-20.408836,-20.884806,-21.933760,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
312,-11.889790,-13.000632,-16.028648,-17.109066,-15.882111,-15.968800,-16.425117,-18.963812,-19.968313,-17.888788,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
313,-25.888529,-22.996035,-22.643534,-22.626450,-21.710609,-22.616358,-21.866781,-19.947834,-19.194817,-20.434992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7


In [24]:
irnewdf.size

332640

In [25]:
df[:]
df3 = pd.DataFrame(df['feature'].values.tolist())
test=df3
newdf = pd.concat([df3,labels], axis=1)
rnewdf = newdf.rename(index=str, columns={"0": "label"})
rnewdf = shuffle(newdf)

In [26]:
rnewdf[:]

,0,1,2,3,4,5,6,7,8,9,...,1046,1047,1048,1049,1050,1051,1052,1053,1054,0
43,-26.613413,-24.101061,-24.044582,-26.273148,-25.615349,-24.027012,-22.242580,-23.689274,-25.741400,-24.539249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
217,-24.362637,-23.627378,-22.561050,-22.656483,-24.527611,-23.993454,-25.036350,-27.185997,-27.938622,-24.186615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
35,-22.106447,-23.103846,-25.356831,-25.696522,-25.006828,-23.222021,-23.218178,-25.083075,-24.683256,-25.530569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
264,-26.065241,-26.354988,-24.760550,-23.114044,-21.957155,-22.302515,-23.463003,-23.129110,-23.218102,-22.639606,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
33,-22.558577,-22.116936,-21.639246,-23.237471,-22.158472,-20.937061,-21.362511,-22.790396,-22.591990,-20.985893,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,-28.691824,-28.018274,-26.784550,-24.531296,-24.096077,-25.961048,-26.064833,-23.557976,-23.144463,-25.022858,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
165,-24.331814,-23.266600,-23.286003,-22.712482,-23.138966,-22.838577,-24.300722,-25.049530,-23.282658,-23.693338,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
276,-25.564301,-22.859564,-21.301481,-23.351416,-24.483458,-23.677341,-23.047371,-23.029810,-22.000204,-21.794502,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
210,-26.748257,-26.450905,-25.290035,-25.369682,-25.898069,-24.254547,-23.604179,-24.882210,-23.224943,-22.891214,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [27]:
rnewdf=rnewdf.fillna(0)

In [28]:
rnewdf

,0,1,2,3,4,5,6,7,8,9,...,1046,1047,1048,1049,1050,1051,1052,1053,1054,0
43,-26.613413,-24.101061,-24.044582,-26.273148,-25.615349,-24.027012,-22.242580,-23.689274,-25.741400,-24.539249,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
217,-24.362637,-23.627378,-22.561050,-22.656483,-24.527611,-23.993454,-25.036350,-27.185997,-27.938622,-24.186615,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
35,-22.106447,-23.103846,-25.356831,-25.696522,-25.006828,-23.222021,-23.218178,-25.083075,-24.683256,-25.530569,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
264,-26.065241,-26.354988,-24.760550,-23.114044,-21.957155,-22.302515,-23.463003,-23.129110,-23.218102,-22.639606,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
33,-22.558577,-22.116936,-21.639246,-23.237471,-22.158472,-20.937061,-21.362511,-22.790396,-22.591990,-20.985893,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,-28.691824,-28.018274,-26.784550,-24.531296,-24.096077,-25.961048,-26.064833,-23.557976,-23.144463,-25.022858,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
165,-24.331814,-23.266600,-23.286003,-22.712482,-23.138966,-22.838577,-24.300722,-25.049530,-23.282658,-23.693338,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
276,-25.564301,-22.859564,-21.301481,-23.351416,-24.483458,-23.677341,-23.047371,-23.029810,-22.000204,-21.794502,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
210,-26.748257,-26.450905,-25.290035,-25.369682,-25.898069,-24.254547,-23.604179,-24.882210,-23.224943,-22.891214,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [29]:
rnewdf.size

332640

In [30]:
train = rnewdf

itrainfeatures = train.iloc[:, :-1]
itrainlabel = train.iloc[:,-2:]

In [31]:
itrainlabel

,1054,0
43,0.0,2
217,0.0,2
35,0.0,1
264,0.0,4
33,0.0,7
...,...,...
289,0.0,2
165,0.0,6
276,0.0,7
210,0.0,1


In [59]:
train = rnewdf

trainfeatures = train.iloc[:, :-1]
trainlabel = test.iloc[:,-2:]


X_train = np.array(trainfeatures)
#y_train = np.array(feeling_list)
y_train = np.array(train.iloc[:,-1:])

y_train = y_train.reshape((315,1))

lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
#y_train = np_utils.to_categorical(y_train)

YY=y_train
XX =np.expand_dims(X_train, axis=2)

D:\Tools\Anaconda\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [60]:
X_train.shape

(315, 1055)

In [61]:
XX.shape

(315, 1055, 1)

In [62]:
YY.shape

(315, 7)

In [63]:
X_train, X_test, Y_train, Y_test = train_test_split(XX,YY, test_size = 0.1)

In [64]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((283, 1055, 1), (32, 1055, 1), (283, 7), (32, 7))

In [65]:
from tensorflow.keras.models import Sequential

In [66]:
#trainX=XX
#trainy=YY
trainX= X_train
trainy= Y_train
testX= X_test
testy= Y_test

n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
model = Sequential()
model.add(LSTM(230, input_shape=(n_timesteps, n_features)))
model.add(Dropout(0.1))
model.add(Dense(230, activation='relu'))
model.add(Dense(230, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

In [67]:
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
opt = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.99, beta_2=0.99, epsilon=None, decay=0, amsgrad=False)
#opt = keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=1e-6, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

In [81]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper_5 (ModuleWrap (None, 230)               213440    
_________________________________________________________________
module_wrapper_6 (ModuleWrap (None, 230)               0         
_________________________________________________________________
module_wrapper_7 (ModuleWrap (None, 230)               53130     
_________________________________________________________________
module_wrapper_8 (ModuleWrap (None, 230)               53130     
_________________________________________________________________
module_wrapper_9 (ModuleWrap (None, 7)                 1617      
Total params: 321,317
Trainable params: 321,317
Non-trainable params: 0
_________________________________________________________________


In [68]:
verbose = 1
epochs = 100
model.fit(trainX, trainy, epochs=epochs,  verbose=verbose, validation_data=(testX, testy))
#score, accuracy = model.evaluate(testX, testy, verbose=verbose)
#print("Test Score: ",score,"   Test Accuracy: ",accuracy)
#print(accuracy)

Epoch 1/100
9/9 [==============================] - 4s 135ms/step - loss: 1.9490 - accuracy: 0.1201 - val_loss: 1.9484 - val_accuracy: 0.1562
Epoch 2/100
9/9 [==============================] - 1s 79ms/step - loss: 1.9429 - accuracy: 0.1519 - val_loss: 1.9500 - val_accuracy: 0.0938
Epoch 3/100
9/9 [==============================] - 1s 78ms/step - loss: 1.9400 - accuracy: 0.1484 - val_loss: 1.9502 - val_accuracy: 0.0625
Epoch 4/100
9/9 [==============================] - 1s 79ms/step - loss: 1.9389 - accuracy: 0.1555 - val_loss: 1.9503 - val_accuracy: 0.0625
Epoch 5/100
9/9 [==============================] - 1s 78ms/step - loss: 1.9387 - accuracy: 0.1555 - val_loss: 1.9506 - val_accuracy: 0.0625
Epoch 6/100
9/9 [==============================] - 1s 77ms/step - loss: 1.9389 - accuracy: 0.1555 - val_loss: 1.9510 - val_accuracy: 0.0625
Epoch 7/100
9/9 [==============================] - 1s 78ms/step - loss: 1.9386 - accuracy: 0.1555 - val_loss: 1.9522 - val_accuracy: 0.0625
Epoch 8/100
9/9 [==